<a href="https://colab.research.google.com/github/GuanRuLai/Python-project-File-RAG-Chatbot/blob/main/Commodity_comment_chatbot_gpt4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install necessary libraries

In [ ]:
!pip install langchain
!pip install langchain_openai rich
!pip install faiss-gpu
!pip install langchain-community
!pip install transformers accelerate
!pip install langchain_experimental
!pip install langchain_huggingface
!pip install unstructured

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: langcha

# Import libraries

In [ ]:
import os
from google.colab import userdata
from google.colab import files
from huggingface_hub import login
# from rich import print as pprint
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.memory import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings

# Set api key

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# Build chat model - Chatgpt

In [ ]:
chat_model = ChatOpenAI(model="gpt-4o")

# Convert text to vector

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Define a function for data processing

In [ ]:
def data_processing(file_path):

  # Define a function for file loading
  def office_file(file_path):
      loader = UnstructuredFileLoader(file_path)
      docs = loader.load()

      return docs

  excel_docs = office_file(file_path)
  # pprint(excel_docs)

  # split the data
  text_splitter = RecursiveCharacterTextSplitter(
      separators=[" ", "\\n", "\n"],
      chunk_size=200,
      chunk_overlap=0)

  splits = text_splitter.split_documents(excel_docs)
  for i in splits[:5]:
      print(i.page_content)
      print("-" * 20)

  return splits

# Define a function for whole RAG chain building

In [ ]:
def rag_from_excel(file_path, name="excel"):
  # Read and process excel file
  splits = data_processing(file_path)

  # Convert text to vector & store into vector database
  db = FAISS.from_documents(splits, embeddings)
  db.save_local("/content/drive/MyDrive/Colab Notebooks/人工智慧-深度學習/深度學習模型實作/生成式 AI/office_db",
                index_name=name)

  new_db = FAISS.load_local(
      folder_path="/content/drive/MyDrive/Colab Notebooks/人工智慧-深度學習/深度學習模型實作/生成式 AI/office_db",
      index_name=name,
      embeddings=embeddings,
      allow_dangerous_deserialization=True)

  # Create retriever
  retriever=new_db.as_retriever()

  tool = create_retriever_tool(
      retriever=retriever,
      name="retriever_tool",
      description="檢索並返回文字相關資訊"
  )

  tools = [tool]

  # Create agent
  agent_prompt = ChatPromptTemplate.from_messages([
      ("system", "你是一位善用工具的好助理, 請仔細判斷上下文來回答問題, 盡量以 excel 檔案內容 RAG 功能優先"),
      MessagesPlaceholder(variable_name="chat_history"),
      ("human", "{input}"),
      MessagesPlaceholder(variable_name="agent_scratchpad")
  ])

  agent = create_openai_tools_agent(llm=chat_model,
                                    tools=tools,
                                    prompt=agent_prompt)

  agent_executor = AgentExecutor(agent=agent,
                                 tools=tools,
                                 verbose=False) # The system won't show whether it use the tool or not

  # Store chat messages history
  memory = SQLChatMessageHistory(
      session_id="test_id",
      connection_string="sqlite:////content/drive/MyDrive/Colab Notebooks/人工智慧-深度學習/深度學習模型實作/生成式 AI/retriever_1.db"
  )

  # Limit the chat memory to the latest six messages for context clarity
  def window_messages(chain_input):
      if len(memory.messages) > 6:
          cur_messages = memory.messages
          memory.clear()
          for message in cur_messages[-6:]:
              memory.add_message(message)
      return None

  # Integrate chat history into the agent's execution for consistent interaction
  def add_history(agent_executor):
      agent_with_chat_history = RunnableWithMessageHistory(
          agent_executor,
          lambda session_id: memory,
          input_messages_key="input",
          history_messages_key="chat_history",
      )
      memory_chain = (
          RunnablePassthrough.assign(messages=window_messages)
          | agent_with_chat_history
      )
      return memory_chain

  return add_history(agent_executor)

# Main program

In [ ]:
flag = True

while flag:
  print("請上傳一個 Excel 文件 (.xlsx)。")
  uploaded = files.upload()

  if not uploaded:
      print("未上傳任何文件，請重試。")
  else:
    try:
      # Read excel file
      file_path = next(iter(uploaded.keys()))
      print(f"文件{file_path}已讀取，您可以開始提問。")

      memory_chain = rag_from_excel(file_path)
      if memory_chain is None:
        print("無法建立記憶鏈，請檢查輸入資料。")
        continue

      while True:
          msg = input("我的問題：")
          if not msg.strip():
              flag = False
              break
          for chunk in memory_chain.stream(
              {"input": msg},
              config={"configurable": {"session_id": "test_id"}}):
              if "output" in chunk:
                  print(f"AI 回覆：{chunk['output']}", end="")

    except Exception as e:
      print(f"發生錯誤：{e}")

請上傳一個 Excel 文件 (.xlsx)。


Saving 商品清單範例.xlsx to 商品清單範例.xlsx
文件商品清單範例.xlsx已讀取，您可以開始提問。
商品名稱 建立日期 內容描述 資料來源 使用者評價 湖光花語擴香瓶 2025-01-09 這是一款以湖光花語擴香瓶為主題的優質產品，設計精緻且用途廣泛。 官網 用戶A: 產品材質粗糙，使用體驗很差。 用戶B: 性價比極低，完全不值得購買。 用戶C: 完全不符合描述，失望至極 暖陽香木精油皂 2024-04-08 這是一款以暖陽香木精油皂為主題的優質產品，設計精緻且用途廣泛。 第三方電商 用戶A:
--------------------
觸感舒適，質量很好。\n用戶B: 用了一陣子效果很好，值得推薦！\n用戶C: 質感很好，推薦給朋友了。 冬之寧靜香氛蠟燭 2024-11-28 這是一款以冬之寧靜香氛蠟燭為主題的優質產品，設計精緻且用途廣泛。 社群推薦 用戶A: 性價比高，滿意的一次購物體驗。\n用戶B: 質感很好，推薦給朋友了。\n用戶C: 包裝精美，送禮很適合。 靜謐竹炭淨化包 2024-12-07
--------------------
這是一款以靜謐竹炭淨化包為主題的優質產品，設計精緻且用途廣泛。 官網 用戶A: 味道很難聞，讓人無法接受。 用戶B: 質感很差，感覺像廉價產品。 用戶C: 效果根本不明顯，浪費錢。 雲朵棉柔毛巾 2024-10-09 這是一款以雲朵棉柔毛巾為主題的優質產品，設計精緻且用途廣泛。 社群推薦 用戶A: 材質一般，感覺不值這個價格。 用戶B: 包裝還不錯，但使用效果很普通。 用戶C: 外觀設計很好。
--------------------
森林綠意沐浴露 2024-02-20 這是一款以森林綠意沐浴露為主題的優質產品，設計精緻且用途廣泛。 官網 用戶A: 用了一陣子效果很好，值得推薦！\n用戶B: 質感很好，推薦給朋友了。\n用戶C: 非常實用，值得推薦。 月光絲滑枕套 2024-07-20 這是一款以月光絲滑枕套為主題的優質產品，設計精緻且用途廣泛。 社群推薦 用戶A: 包裝精美，送禮很適合。\n用戶B:
--------------------
氣味清新自然，超喜歡！\n用戶C: 用了一陣子效果很好，值得推薦！
--------------------
我的問題：我要關於冬之寧靜香氛蠟燭的商品資訊
AI 回